# **Global settings**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import rcParams
from matplotlib.pyplot import rc_context
import scanpy as sc
import scrublet as scr
import scvelo as scv
import seaborn as sns
import scipy.io
import os
import dotplot
import dotplot.utils
import math
import gseapy as gp
from gseapy.plot import barplot, dotplot, gseaplot
from gseapy.scipalette import SciPalette
from pylab import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap 

In [ ]:
sc.settings.set_figure_params(dpi=100, dpi_save=300, figsize=(5, 5))

In [ ]:
plt.set_cmap('viridis')

<Figure size 500x500 with 0 Axes>

In [ ]:
scv.set_figure_params()

In [ ]:
os.chdir('/disk213/xieqq/JINHUA138.sc')

## **RNA velocity (cell lineage)**

In [ ]:
adata=sc.read_h5ad('adata_CellLineage_rank_genes_groups.h5ad')

In [ ]:
os.chdir('/disk213/xieqq/JINHUA138.sc/velocyto')

In [ ]:
PATH='/disk213/xieqq/JINHUA138.sc/velocyto/'
ap={}
al={}
sample=['CE_0','CE_60','CE_90','CE_180','CE_240',
        'CO_0','CO_60','CO_90','CO_180','CO_240',
        'DU_0','DU_60','DU_90','DU_180','DU_240',
        'IL_0','IL_60','IL_90','IL_180','IL_240',
        'JE_0','JE_60','JE_90','JE_180','JE_240']

for i in sample:
    ap[f'{i}']=sc.read_loom(PATH+f'{i}'+'/'+f'{i}'+'.loom',sparse=True)
    ap[f'{i}'].var_names_make_unique
    length=len(f'{i}')+1
    ap[f'{i}'].obs.index=ap[f'{i}'].obs.index.str[length:-1]
    x=['{}','1']
    ap[f'{i}'].obs.index=ap[f'{i}'].obs.index.map('-'.join(x).format)
    y=['{}',f'{i}']
    ap[f'{i}'].obs.index=ap[f'{i}'].obs.index.map('_'.join(y).format) #每个barcode后加上sample名
    if(ap[f'{i}'].obs.shape[1]!=0):
        ap[f'{i}'].obs=ap[f'{i}'].obs.drop(columns=['Clusters','_X', '_Y'])
    al[f'{i}']=scv.utils.merge(adata[adata.obs['PRO1_JH'].isin([f'{i}'])], ap[f'{i}'])

In [ ]:
ldata=sc.concat(ap.values(),keys=ap.keys(),label='loom_JH')
ldata

In [ ]:
ldata.write('velocyto_merge.h5ad')

In [ ]:
adata=scv.utils.merge(adata, ldata)
adata

In [ ]:
adata.write('adata_velocyto_merge.h5ad')

In [ ]:
#scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000)
scv.pp.moments(adata)

In [ ]:
scv.tl.velocity(adata)

computing velocities
    finished (0:05:50) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)


In [ ]:
scv.tl.velocity_graph(adata)

In [ ]:
scv.pl.velocity_embedding_grid(adata, basis='umap', color='CellLineage_change_categories')
scv.pl.velocity_embedding_stream(adata, basis='umap', color='CellLineage_change_categories', legend_loc='right margin')

In [ ]:
# adata.write('adata_CellLineage_velocity_merge.h5ad')

In [ ]:
PATH='/disk213/xieqq/JINHUA138.sc/velocyto/'
sample=['CE_0','CE_60','CE_90','CE_180','CE_240',
        'CO_0','CO_60','CO_90','CO_180','CO_240',
        'DU_0','DU_60','DU_90','DU_180','DU_240',
        'IL_0','IL_60','IL_90','IL_180','IL_240',
        'JE_0','JE_60','JE_90','JE_180','JE_240']

for i in sample:
    al[f'{i}'].write(PATH+f'{i}'+'/'+'CellLineage_velocyto_'+f'{i}'+'.h5ad')

In [ ]:
for i in sample:
    newdata=al[f'{i}']
    scv.pp.moments(newdata)
    scv.tl.velocity(newdata)
    scv.tl.velocity_graph(newdata)
    scv.pl.velocity_embedding_grid(newdata, basis='umap', arrow_length=3, arrow_size=2, color='CellLineage_change_categories', show=False, save='grid_CellLineage_'+f'{i}'+'.pdf') 
    scv.pl.velocity_embedding_grid(newdata, basis='umap', arrow_length=3, arrow_size=2, color='CellLineage_change_categories', alpha=0, show=False, save='grid_CellLineage_alpha0_'+f'{i}'+'.pdf') #网格线

## **RNA velocity**

In [ ]:
Epithelial=sc.read_h5ad('Epithelial_CellType.h5ad')

In [ ]:
os.chdir('/disk213/xieqq/JINHUA138.sc/velocyto')

In [ ]:
PATH='/disk213/xieqq/JINHUA138.sc/velocyto/'
ap={}
al={}
sample=['CE_0','CE_60','CE_90','CE_180','CE_240',
        'CO_0','CO_60','CO_90','CO_180','CO_240',
        'DU_0','DU_60','DU_90','DU_180','DU_240',
        'IL_0','IL_60','IL_90','IL_180','IL_240',
        'JE_0','JE_60','JE_90','JE_180','JE_240']

for i in sample:
    ap[f'{i}']=sc.read_loom(PATH+f'{i}'+'/'+f'{i}'+'.loom',sparse=True)
    ap[f'{i}'].var_names_make_unique
    length=len(f'{i}')+1
    ap[f'{i}'].obs.index=ap[f'{i}'].obs.index.str[length:-1]
    x=['{}','1']
    ap[f'{i}'].obs.index=ap[f'{i}'].obs.index.map('-'.join(x).format)
    y=['{}',f'{i}']
    ap[f'{i}'].obs.index=ap[f'{i}'].obs.index.map('_'.join(y).format) #每个barcode后加上sample名
    if(ap[f'{i}'].obs.shape[1]!=0):
        ap[f'{i}'].obs=ap[f'{i}'].obs.drop(columns=['Clusters','_X', '_Y'])
    al[f'{i}']=scv.utils.merge(Epithelial[Epithelial.obs['PRO1_JH'].isin([f'{i}'])], ap[f'{i}'])

In [ ]:
ldata=sc.concat(ap.values(),keys=ap.keys(),label='loom_JH')
ldata

In [ ]:
# ldata.write('velocyto_merge.h5ad') 

In [ ]:
Epithelial=scv.utils.merge(Epithelial, ldata)
Epithelial

In [ ]:
Epithelial.write('Epithelial_velocyto_merge.h5ad')

In [ ]:
#scv.pp.filter_and_normalize(Epithelial, min_shared_counts=20, n_top_genes=2000)
scv.pp.moments(Epithelial)

In [ ]:
scv.tl.velocity(Epithelial)

In [ ]:
scv.tl.velocity_graph(Epithelial)

In [ ]:
scv.pl.velocity_embedding_stream(Epithelial, basis='umap', color='Epithelial_change_categories', legend_loc='right margin')

In [ ]:
PATH='/disk213/xieqq/JINHUA138.sc/velocyto/'
sample=['CE_0','CE_60','CE_90','CE_180','CE_240',
        'CO_0','CO_60','CO_90','CO_180','CO_240',
        'DU_0','DU_60','DU_90','DU_180','DU_240',
        'IL_0','IL_60','IL_90','IL_180','IL_240',
        'JE_0','JE_60','JE_90','JE_180','JE_240']

for i in sample:
    al[f'{i}'].write(PATH+f'{i}'+'/'+'Epithelial_velocyto_'+f'{i}'+'.h5ad')

In [ ]:
for i in sample:
    newdata=al[f'{i}']
    scv.pp.moments(newdata)
    scv.tl.velocity(newdata)
    scv.tl.velocity_graph(newdata)
    scv.pl.velocity_embedding_grid(newdata, basis='umap', arrow_length=3, arrow_size=2, color='Epithelial_change_categories', show=False, save='grid_Epithelial_'+f'{i}'+'.pdf') 
    scv.pl.velocity_embedding_grid(newdata, basis='umap', arrow_length=3, arrow_size=2, color='Epithelial_change_categories', alpha=0, show=False, save='grid_Epithelial_alpha0_'+f'{i}'+'.pdf') #网格线